In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Anxiety_disorder/GSE119995'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Exposure-induced changes of plasma mRNA expression levels in patients with panic disorder"
!Series_summary	"Anxiety disorders including panic disorders with or without agoraphobia are the most prevalent mental disorders. Exposure is a core technique within the framework of cognitive behavioral therapy to treat phobia- and anxiety-related symptoms. The primary aim of this study was to trace specific anxiety-related plasma gene expression changes of subjects with PD at three time points in order to identify biomarkers for acute anxiety states. In this intervention, the patient is exposed to highly feared and mostly avoided situations."
!Series_overall_design	"Blood samples from individuals with panic disorder (n=24) were drawn at three time points during exposure: baseline, 1 hour post-exposure and 24 hours after exposure-onset."
Sample Characteristics Dictionary:
{0: ['disease: panic disorder'], 1: ['tissue: blood plasma'], 2: ['Sex: female', 'Sex:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable


### Step 1: Gene Expression Data Availability: Unknown Step

In [4]:
is_gene_available = True  # Based on the background information provided, gene expression data appears to be available.


### Step 2: Variable Availability and Data Type Conversion: Unknown Step

In [5]:
# Anxiety Disorder
trait_row = 0  # inferred from 'disease: panic disorder'
# Age: not available in the dataset provided
age_row = None
# Gender
gender_row = 2  # inferred from 'Sex: female', 'Sex: male', 'Sex: not determined'

# Data Type Conversion Functions
def convert_trait(value):
    return 1 if 'panic disorder' in value else 0

def convert_age(value):
    try:
        return float(value.split(':')[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    value = value.split(':')[1].strip().lower()
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None

# Save Metadata
save_cohort_info('GSE119995', './preprocessed/Anxiety_disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(
        clinical_data, 'Anxiety_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender
    )
    csv_path = './preprocessed/Anxiety_disorder/trait_data/GSE119995.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3391438': [1, 0], 'GSM3391439': [1, 0], 'GSM3391440': [1, 0], 'GSM3391441': [1, 0], 'GSM3391442': [1, 0], 'GSM3391443': [1, 0], 'GSM3391444': [1, 1], 'GSM3391445': [1, 1], 'GSM3391446': [1, 1], 'GSM3391447': [1, 0], 'GSM3391448': [1, 0], 'GSM3391449': [1, 0], 'GSM3391450': [1, 1], 'GSM3391451': [1, 1], 'GSM3391452': [1, 1], 'GSM3391453': [1, 0], 'GSM3391454': [1, 0], 'GSM3391455': [1, 0], 'GSM3391456': [1, 0], 'GSM3391457': [1, 0], 'GSM3391458': [1, 0], 'GSM3391459': [1, 0], 'GSM3391460': [1, 0], 'GSM3391461': [1, 0], 'GSM3391462': [1, 0], 'GSM3391463': [1, 0], 'GSM3391464': [1, 0], 'GSM3391465': [1, 1], 'GSM3391466': [1, 1], 'GSM3391467': [1, 1], 'GSM3391468': [1, 0], 'GSM3391469': [1, 0], 'GSM3391470': [1, 0], 'GSM3391471': [1, 0], 'GSM3391472': [1, 0], 'GSM3391473': [1, 0], 'GSM3391474': [1, 0], 'GSM3391475': [1, 0], 'GSM3391476': [1, 0], 'GSM3391477': [1, 0], 'GSM3391478': [1, 0], 'GSM3391479': [1, 0], 'GSM3391480': [1, 1], 'GSM3391481': [1, 1], 'GSM3391482': [1, 1], 'GSM33914

### Step 3: Gene Data Extraction

In [6]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [7]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [8]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [9]:
# 1. Identify the correct keys for identifier and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [10]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Anxiety_disorder/gene_data/GSE119995.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Anxiety_disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE119995', './preprocessed/Anxiety_disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Anxiety_disorder/GSE119995.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Anxiety_disorder':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Anxiety_disorder' in this dataset is severely biased.

For the feature 'Gender', the least common label is '1.0' with 18 occurrences. This represents 28.57% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

